In [1]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [2]:
%load_ext autoreload
%autoreload 2

from utils import helper, config, rayer
from ml.models.base.HyperOpt.brisk_bagging import BriskBagging

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common
import ray
import time
import pandas as pd

In [3]:
rayer.get_global_cluster(num_cpus=10)

In [4]:
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-24 08:27:55.703459 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 3 small-group
Pending:
 10.10.86.102: small-group, waiting
 10.10.96.177: small-group, waiting
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/7.0 CPU
 0.00/24.214 GiB memory
 0.00/7.114 GiB object_store_memory

Demands:
 {}: 1+ pending tasks/actors
 {'CPU': 1}: 10+ from request_resources()


In [5]:
X, y = helper.get_covid_dataset()
X = X.drop(['location'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

ss = StandardScaler()
X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns = X_test.columns)

In [6]:
X_train_id = ray.put(X_train)
y_train_id = ray.put(y_train)
X_test_id = ray.put(X_test)
y_test_id = ray.put(y_test)

In [7]:

@ray.remote(num_returns=1)
def worker(base_model, X_train, X_test, y_train, y_test):     
    base_model.fetch_model(X_train, X_test, y_train, y_test)
    return base_model


In [8]:
brisk_bag_1 = BriskBagging('brisk_xgb1')
brisk_bag_1.n_estimators = 100


brisk_bag_2 = BriskBagging('brisk_xgb2')
brisk_bag_2.n_estimators = 100

list_models = [brisk_bag_1, brisk_bag_2]

In [9]:
model_results = ray.get([worker.remote(base_model, X_train_id, X_test_id, y_train_id, y_test_id) for base_model in list_models])

80                                                     
  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]


(worker pid=927) 2022-11-24T06:28:04PST : INFO : brisk_bagging : __discover_model__ : 96 : Message : brisk_xgb1: Starting training for trials:300, n_estimators  100
(worker pid=927) 2022-11-24T06:28:04PST : INFO : tpe : suggest : 864 : Message : build_posterior_wrapper took 0.000438 seconds
(worker pid=927) 2022-11-24T06:28:04PST : INFO : tpe : suggest : 900 : Message : TPE using 0 trials


84                                                                              
  1%|          | 1/100 [00:00<00:30,  3.27trial/s, best loss: 2008464.305749236]


(worker pid=927) 2022-11-24T06:28:04PST : INFO : tpe : suggest : 864 : Message : build_posterior_wrapper took 0.000370 seconds
(worker pid=927) 2022-11-24T06:28:04PST : INFO : tpe : suggest : 900 : Message : TPE using 1/1 trials with best loss 2008464.305749


91                                                                               
  2%|▏         | 2/100 [00:00<00:26,  3.67trial/s, best loss: 1975585.7808430146]
75                                                                               
  3%|▎         | 3/100 [00:00<00:25,  3.85trial/s, best loss: 1943688.722284866]


...

(worker pid=220, ip=10.10.63.163) 2022-11-24T06:29:32PST : INFO : tpe : suggest : 864 : Message : build_posterior_wrapper took 0.000251 seconds
(worker pid=220, ip=10.10.63.163) 2022-11-24T06:29:32PST : INFO : tpe : suggest : 900 : Message : TPE using 96/96 trials with best loss 1767843.865914
(worker pid=220, ip=10.10.63.163) 2022-11-24T06:29:32PST : INFO : tpe : suggest : 864 : Message : build_posterior_wrapper took 0.000275 seconds
(worker pid=220, ip=10.10.63.163) 2022-11-24T06:29:32PST : INFO : tpe : suggest : 900 : Message : TPE using 97/97 trials with best loss 1767843.865914
(worker pid=220, ip=10.10.63.163) 2022-11-24T06:29:32PST : INFO : tpe : suggest : 864 : Message : build_posterior_wrapper took 0.000268 seconds
(worker pid=220, ip=10.10.63.163) 2022-11-24T06:29:32PST : INFO : tpe : suggest : 900 : Message : TPE using 98/98 trials with best loss 1767843.865914


93                                                                                
 99%|█████████▉| 99/100 [00:09<00:00, 10.91trial/s, best loss: 1767843.8659135439]


(worker pid=220, ip=10.10.63.163) 2022-11-24T06:29:32PST : INFO : tpe : suggest : 864 : Message : build_posterior_wrapper took 0.000266 seconds
(worker pid=220, ip=10.10.63.163) 2022-11-24T06:29:32PST : INFO : tpe : suggest : 900 : Message : TPE using 99/99 trials with best loss 1767843.865914


100%|██████████| 100/100 [00:09<00:00, 10.57trial/s, best loss: 1767843.8659135439]


(worker pid=220, ip=10.10.63.163) 2022-11-24T06:29:33PST : INFO : brisk_bagging : refit : 146 : Message : final score :[0.9346682979354216, 0.6738857230000795]


In [11]:
model_results